In [1]:
import os
import time
import getpass
import glob
import ipywidgets as widgets
import scripts.filter_closing_smm as cfilter
import subprocess
import sys
from ipywidgets import VBox, HBox, Label, interactive, Box, HTML, fixed, interact, TwoByTwoLayout
from tqdm.notebook import trange, tqdm

In [2]:
from tqdm.notebook import 

SyntaxError: invalid syntax (<ipython-input-2-15dd9eb8b03f>, line 1)

In [ ]:
from tqdm.notebook import trange, tqdm

In [ ]:
user = getpass.getuser()
raw_path = f'/home/{user}/pysmm_downloads/0_raw/'
processed_path = raw_path.replace('0_raw','1_processed')
cores = os.cpu_count()

In [ ]:
desc_postprocess =  """<p style="line-height: 20px">After the data is filtered, a time series analysis of the soil moisture maps can be performed. Several statistics can be applied whether to the entire time series or to a specified range, statistics as median, mean, standard deviation or linear trend (slope of the line) are available to process the selected data.  </br>The slope of the linear trend, indicates if the trend in soil moisture is negative or positive. These trends might be related to peatland management practices. 
After the processing completes, download the outputs and check them in a GIS environment such as QGIS or ArcGIS.</p>"""

In [ ]:
def return_paths(raw_path):
    
    """Create a list of folders in a given path
    skipping those with begin with '.' and are empty

    """
    paths = [folder for folder in os.listdir(raw_path) 
             if os.path.isdir(os.path.join(raw_path, folder)) and not folder.startswith('.') 
             and len(os.listdir(os.path.join(raw_path, folder))) != 0
    ]
    paths.sort()
    return paths

def get_select2(*args):
    options = return_paths(os.path.join(raw_path, select_1.value))
    select_2.options = options

def closing_filter(process_path):
    
    IMAGES_TYPES = ('.tif')
    folder = process_path
    image_files = []
    if os.path.isdir(folder):
        for root, dirs, files in os.walk(folder):
            if len(files) != 0:
                files = [os.path.join(root, x) for x in files if x.endswith(IMAGES_TYPES)]
                [image_files.append(os.path.abspath(file)) for file in files]
        image_files.sort()
    
    else:
        print(f'ERROR: The {folder} joined is not a directory path.')
    print(f'There are {len(image_files)} images to process, please wait...')

    for i in trange(len(image_files)):
        cfilter.raw_to_processed(image_files[i])

    return 0

def time_message(stat, cores):
    output_msg = f'Computing {stat.lower()} using {cores} cores, please wait...'
    if stat == 'linear_trend':
        lt_msg = f'\nDepending on the extent and the number of images, this process could take several minutes...'
        return "".join((output_msg, lt_msg))
    else:
        return output_msg

def stack_composed(statistic, feature, field, cores, ini_date=None, end_date=None):
    
    cores = str(cores) # get the number of cores of the instance
    processed_ff_path = os.path.join(processed_path, feature, field)
    processed_stat_path = os.path.join(processed_ff_path, 'stats')

    if not os.path.exists(processed_stat_path):
         os.makedirs(processed_stat_path)
    
    tifs = glob.glob(f'{processed_ff_path}/close*.tif')
    
    print(time_message(statistic, cores))
    
    tic = time.perf_counter()
    if ini_date:
        start = ini_date.strftime("%Y-%m-%d")
        end = end_date.strftime("%Y-%m-%d")
        
        processed_stat_name = os.path.join(processed_stat_path, 
                                           f'Stack_{statistic}_{feature}_{field}_{start}_{end}.tif')
        
        process = subprocess.run(['python3', f'{os.getcwd()}/scripts/stackcomposed/bin/stack-composed',
                                    '-stat', statistic,
                                    '-bands', '1',
                                    '-start', start,
                                    '-end', end,
                                    '-p', cores,
                                    '-o', processed_stat_name]+
                                    tifs,
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE,
                                universal_newlines=True)
    else:
        processed_stat_name = os.path.join(processed_stat_path, 
                                           f'Stack_{statistic}_{feature}_{field}.tif')
        process = subprocess.run(['python3', f'{os.getcwd()}/scripts/stackcomposed/bin/stack-composed',
                                    '-stat', statistic,
                                    '-bands', '1',
                                    '-p', cores,
                                    '-o', processed_stat_name]+
                                    tifs,
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE,
                                universal_newlines=True)
        
    if process.returncode == 0:
        toc = time.perf_counter()
        elapsed_time = round(toc-tic, 2)
        print(f'Done in {elapsed_time} seconds!')
        link = f"https://sepal.io/api/sandbox/jupyter/tree/pysmm_downloads/1_processed/{feature}/{field}"
        html_link = f'<br>The processed images can be found in <a href = "{link}" target="_blank"> {processed_ff_path}</a>'
        display(HTML(html_link))
    else:
        print('MemoryError: Unable to allocate memory for the process, \
please consider increase the RAM memory by changing the instance.')
        print(process.stderr)
        
def run_stat_all_ts(statistic, feature, field, cores):
    stack_composed(statistic, feature, field, cores)

def run_stat_range(statistic, feature, field, 
                  cores, ini_date, end_date):
    stack_composed(statistic, feature, field, cores, ini_date, end_date)

def stats_span(feature, field):
    kwargs = {
        'feature' : fixed(feature),
        'field': fixed(field),
        'statistic' : statistic,
        'cores':w_cores
    }
    
    button_layout = widgets.Layout(
        width='175px'
    )
    
    # Tab to entire series
    
    w_ts = interactive(run_stat_all_ts, {'manual':True}, **kwargs)
    w_ts_button = w_ts.children[-2]
    w_ts_button.layout = button_layout
    w_ts_button.button_style = 'info'
    w_ts_button.icon = 'check'
    w_ts_button.description = 'Compute time series!'
    w_ts_output = VBox([HBox([Label('Select the statistic you want compute:'), 
                            VBox([w_ts.children[0], w_ts.children[1]]), w_ts_button, 
                        ]), w_ts.children[-1]
                    ])

    # Tab to range 
    
    kwargs['ini_date'] = ini_date
    kwargs['end_date'] = end_date
    w_range = interactive(run_stat_range, {'manual':True}, **kwargs)
    w_range_button = w_range.children[-2]
    w_range_button.layout = button_layout
    w_range_button.button_style = 'info'
    w_range_button.icon = 'check'
    w_range_button.description = 'Compute range!'
    w_range_output = VBox([HBox([Label('Select the statistic you want compute:'),  
                            VBox([w_range.children[0], w_range.children[2], w_range.children[3], w_range.children[1]]),
                            w_range_button, 
                            ]),
                           w_range.children[-1]
                    ]) 
    # Tabs 
    
    children = [w_ts_output, w_range_output]
    
    tab = widgets.Tab()
    tab.children = children
    tab.set_title(0, 'All time series')
    tab.set_title(1, 'Range')
    
    display(tab)
        
def filter_process(feature, field):
    
    print('Postprocessing the soil moisture maps')
    process_path = os.path.join(raw_path, feature, field)
    closing_filter(process_path)
    display(HTML(desc_postprocess))
    stats_span(feature, field)

def run():
    w = interactive(filter_process, {'manual':True}, feature=select_1, field=select_2)
    w.children[0].description = ''
    w.children[1].description = ''
    run_button = w.children[2]
    run_button.description = 'Process the folder'
    run_button.button_style = 'info'
    run_button.icon='check'
    display(HBox([Label('Select the folder you want process:'), w.children[0], w.children[1]]))
    display(HBox([w.children[2]]))
    display(w.children[-1])
     

raw_folders = []
    


In [ ]:
statistic = widgets.Dropdown(
    options=[('Median','median'), ('Mean','mean'), ('Gmean','Gmean'), 
             ('Max','max'), ('Min','min'), ('Std','std'), 
             ('Valid pixels','valid_pixels'), 
             ('Linear trend','linear_trend')],
    value='median',
    disabled=False,
)

time_span = widgets.ToggleButtons(
    options=['All time series', 'Range'],
    description='Dates:',
    disabled=False
)

ini_date = widgets.DatePicker(
    description='Start date',
    disabled=False
)

end_date = widgets.DatePicker(
    description='End date',
    disabled=False
)

time_span = widgets.ToggleButtons(
    options=['All time series', 'Range'],
    description='Dates:',
    disabled=False
)

ini_date = widgets.DatePicker(
    description='Start date',
    disabled=False
)

end_date = widgets.DatePicker(
    description='End date',
    disabled=False
)

select_1 = widgets.Select(
    options=return_paths(raw_path),
)

w_cores = widgets.IntSlider(
    value=cores,
    min=1,
    max=cores,
    step=1,
    description='Processors:',
    orientation='horizontal', 
    readout=True
)

select_2 = widgets.Select()
select_1.observe(get_select2)    
    

In [ ]:
run()